In [4]:
import pandas as pd
import numpy as np
from collections import deque
import heapq

### 'Manhattan" distance between two points in the graph. Diff from Euclidean distance
$$|x_1-x_2| + |y_1-y_2|$$

In [5]:
def distance(pos1, pos2):
  return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

### our heuristic is 
$$h(n) = |n_{\text{row}} - g_{\text{row}}| + |n_{\text{col}} + g_{\text{col}}|$$
#### where $n_{\text{row}} \text{ and } n_{\text{col}} \text{ is the current position, and } g_{\text{row}} \text{ and } g_{\text{col}}$ are goal coordinates
* our heuristic is consistent as moving from one square to another adjacent square decreases remaining distance to goal coordinates by 1

In [32]:
def pathfinding(filepath):
    df = pd.read_csv(filepath)
    size = df.shape
    print(df.head(), "\n")

    m, n = size[0], size[1]

    start, goal = (0, 0), (0, 0)

    def find_start_and_goal_cell() -> tuple[[tuple[int, int], tuple[int, int]]]:
        nonlocal start, goal
        for i, row in df.iterrows():
            for j, n in enumerate(row):
                if n == "S":
                    start = (i, j)
                if n == "G":
                    goal = (i, j)
        return start, goal
    find_start_and_goal_cell()

    def heuristic(pos) -> int:
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

    class State:
        def __init__(self, pos, cur_keys, doors_opened, keys_collected):
            self.pos = pos
            self.cur_keys = cur_keys
            self.doors_opened = frozenset(doors_opened)
            self.keys_collected = frozenset(keys_collected)

        def __eq__(self, __o: object) -> bool:
            return (
                self.pos == __o.pos
                and self.cur_keys == __o.cur_keys
                and self.doors_opened == __o.doors_opened
                and self.keys_collected == __o.keys_collected
            )

        def __hash__(self) -> int:
            return hash(
                (self.pos, self.cur_keys, self.doors_opened, self.keys_collected)
            )

    start_state = State(start, 0, frozenset(), frozenset())
    heap = []
    counter = 0
    heapq.heappush(heap, (heuristic(start), 0, counter, start_state))
    counter += 1

    g_score = {start_state: 0}
    came_from = {}
    explored = set()
    states_explored = 0

    while heap:
        heuristic_val, current_cost, counter, current_state = heapq.heappop(heap)

        if current_state.pos == goal:
            optimal_path = []
            state = current_state
            while state in came_from:
                optimal_path.append(state.pos)
                state = came_from[state]
            optimal_path.append(start_state.pos)
            optimal_path.reverse()
            optimal_cost = g_score[current_state]
            return optimal_path, optimal_cost, states_explored
        if current_state in explored:
            continue
        explored.add(current_state)
        states_explored += 1

        row, col = current_state.pos
        dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for d_row, d_col in dirs:
            new_row, new_col = row + d_row, col + d_col
            if 0 <= new_row < m and 0 <= new_col < n:
                cell = df.iloc[new_row, new_col]
                new_pos = (new_row, new_col)
                new_keys_in_possession = current_state.cur_keys
                new_doors_opened = set(current_state.doors_opened)
                new_keys_collected = set(current_state.keys_collected)
                can_move = False

                if cell in ("O", "S", "G"):
                    can_move = True
                elif cell == "K":
                    can_move = True
                    if new_pos not in current_state.keys_collected:
                        new_keys_in_possession += 1
                        new_keys_collected.add(new_pos)
                elif cell == "D":
                    if new_pos in current_state.doors_opened:
                        can_move = True
                    elif current_state.cur_keys >= 1:
                        can_move = True
                        new_keys_in_possession -= 1
                        new_doors_opened.add(new_pos)
                    else:
                        can_move = False
                else:
                    can_move = False

                if can_move:
                    neighbor_state = State(
                        new_pos,
                        new_keys_in_possession,
                        frozenset(new_doors_opened),
                        frozenset(new_keys_collected),
                    )
                    new_g_score = g_score[current_state] + 1
                    if neighbor_state in explored:
                        continue
                    if (
                        neighbor_state not in g_score
                        or new_g_score
                        or new_g_score < g_score[neighbor_state]
                    ):
                        came_from[neighbor_state] = current_state
                        g_score[neighbor_state] = new_g_score
                        f_score = new_g_score + heuristic(new_pos)
                        heapq.heappush(heap, (f_score, new_g_score, counter, neighbor_state))
                        counter += 1

    return [], -1, states_explored

In [33]:
pathfinding("data/E0/grid.csv")

   S  K  O
0  D  D  O
1  G  O  O 



([(0, 0), (1, 0)], 1, 1)

In [34]:
pathfinding("data/E1/grid.csv")

   S  O
0  K  O
1  D  O
2  K  O
3  K  O
4  D  O 



([(0, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (6, 0)],
 8,
 20)

In [35]:
pathfinding("data/E2/grid.csv")

   S  D D.1  O O.1
0  K  D   D  D   O
1  D  D   D  D   O
2  K  D   D  D   O
3  K  D   D  D   O
4  D  D   D  D   O 



([], -1, 1)

In [36]:
pathfinding("data/E3/grid.csv")

Empty DataFrame
Columns: [S, K, K.1, K.2, K.3, D, D.1, D.2, D.3, G]
Index: [] 



([(0, 0)], 0, 0)